<a href="https://colab.research.google.com/github/Ovi-Poddar/CSE-316-Project-Alarm-Clock/blob/main/Image_Caption_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pushing the notebook to the repository
---

After making changes to your notebook, please commit and push them to the repository.
1. To do so from within a Colab notebook, click `File → Save a copy in GitHub`
2. You will be prompted to add a commit message, and after you click OK, the notebook will be pushed to your repository
3. Please push it to [GitHub - rng70/Image-Caption-Generation: image caption generation using rnn and cnn](https://github.com/rng70/Image-Caption-Generation)


# Preprocessing

### Mount Google Drive
---

We need dataset before any kind of analysis. Here, data from google drive is mounted for making dataset

* Folder Structure: <br>
    &nbsp;&nbsp;&nbsp;&nbsp;/My Drive/flickr8k-dataset/{dataset-directory-name}
    <br>
    <br>
* Available Dataset: <br>
    01. &nbsp;&nbsp;&nbsp;&nbsp;Flickr8k_Dataset/
    02. &nbsp;&nbsp;&nbsp;&nbsp;Flickr8k_text/


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

# Dataset Preperation

In [ ]:
# check python version
!python --version

In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader,Dataset
import torchvision.transforms as T

In [ ]:
import numpy as np


def getWindows(input, output_size, kernel_size, padding=0, stride=1, dilate=0):
    working_input = input
    working_pad = padding
    # dilate the input if necessary
    if dilate != 0:
        working_input = np.insert(working_input, range(1, input.shape[2]), 0, axis=2)
        working_input = np.insert(working_input, range(1, input.shape[3]), 0, axis=3)

    # pad the input if necessary
    if working_pad != 0:
        working_input = np.pad(working_input, pad_width=((0,), (0,), (working_pad,), (working_pad,)), mode='constant', constant_values=(0.,))

    in_b, in_c, out_h, out_w = output_size
    out_b, out_c, _, _ = input.shape
    batch_str, channel_str, kern_h_str, kern_w_str = working_input.strides
    

    return np.lib.stride_tricks.as_strided(
        working_input,
        (out_b, out_c, out_h, out_w, kernel_size, kernel_size),
        (batch_str, channel_str, stride * kern_h_str, stride * kern_w_str, kern_h_str, kern_w_str)
    )


class Conv2D:
    """
    An implementation of the convolutional layer. We convolve the input with out_channels different filters
    and each filter spans all channels in the input.
    """
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=0):
        """
        :param in_channels: the number of channels of the input data
        :param out_channels: the number of channels of the output(aka the number of filters applied in the layer)
        :param kernel_size: the specified size of the kernel(both height and width)
        :param stride: the stride of convolution
        :param padding: the size of padding. Pad zeros to the input with padding size.
        """
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding

        self.cache = None

        self._init_weights()

    def _init_weights(self):
        self.weight = 1e-3 * np.random.randn(self.out_channels, self.in_channels,  self.kernel_size, self.kernel_size)
        self.bias = np.zeros(self.out_channels)

    def forward(self, x):
        """
        The forward pass of convolution
        :param x: input data of shape (N, C, H, W)
        :return: output data of shape (N, self.out_channels, H', W') where H' and W' are determined by the convolution
                 parameters.
        """

        n, c, h, w = x.shape
        out_h = (h - self.kernel_size + 2 * self.padding) // self.stride + 1
        out_w = (w - self.kernel_size + 2 * self.padding) // self.stride + 1

        windows = getWindows(x, (n, c, out_h, out_w), self.kernel_size, self.padding, self.stride)
        for window in windows:
          print("==============================================================")
          print("Fucking Windowds")
          print(window)
          print("==============================================================")

        out = np.einsum('bihwkl,oikl->bohw', windows, self.weight)

        # add bias to kernels
        out += self.bias[None, :, None, None]

        self.cache = x, windows
        return out

    def backward(self, dout):
        """
        The backward pass of convolution
        :param dout: upstream gradients
        :return: dx, dw, and db relative to this module
        """
        x, windows = self.cache

        padding = self.kernel_size - 1 if self.padding == 0 else self.padding

        dout_windows = getWindows(dout, x.shape, self.kernel_size, padding=padding, stride=1, dilate=self.stride - 1)
        rot_kern = np.rot90(self.weight, 2, axes=(2, 3))

        db = np.sum(dout, axis=(0, 2, 3))
        dw = np.einsum('bihwkl,bohw->oikl', windows, dout)
        dx = np.einsum('bohwkl,oikl->bihw', dout_windows, rot_kern)

        return db, dw, 

In [ ]:
in_channels = 1
out_channels = 128
kernel_size = 3
stride = 2
padding = 1
batch_size = (4, in_channels, 12, 10)  # expected input size
dout_size = (4, out_channels, 6, 5)   # expected to match the size of the layer outputs

np.random.seed(42)  # for reproducibility

In [ ]:
x = np.random.random(batch_size)  # create data for forward pass
dout = np.random.random(dout_size)  # create random data for backward
#print(x)
print('x: ', x.shape)
#print('d_out: ', dout.shape)

In [ ]:
conv = Conv2D(in_channels, out_channels, kernel_size, stride, padding)

In [ ]:
conv_out = conv.forward(x)
# db, dw, dx = conv.backward(dout)

print('conv_out: ', conv_out.shape)


# Set Up

In [ ]:
!pip install -U tensorflow_text tensorflow tensorflow_datasets

In [ ]:
!pip install einops

# Import Libraries

In [ ]:
import concurrent.futures
import collections
import dataclasses
import hashlib
import itertools
import json
import math
import os
import pathlib
import random
import re
import string
import time
import urllib.request

import einops
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
import requests
import tqdm

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow_datasets as tfds

# Load Data

In [ ]:
# Define the path to the Flickr8k dataset in your Google Drive
flickr8k_dir = '/content/drive/MyDrive/flickr8k-dataset/flickr8k'

## Load Images

In [ ]:
import glob
image_dir = os.path.join(flickr8k_dir, 'Flickr8k_Images')
image_files = glob.glob(image_dir + "/*") # Get a list of all image file paths

In [ ]:
print(len(image_files))

## Load Captions

In [ ]:
# Load the captions for the Flickr8k dataset
captions_filename = os.path.join(flickr8k_dir, 'Flickr8k_Captions', 'captions.txt')
captions_file = open(captions_filename, 'r')
captions = captions_file.readlines()[1:]
captions_file.close()

In [ ]:
# Create a DataFrame to store the captions
captions_df = pd.DataFrame(columns=['image_id', 'caption'])

# Extract the image ID and caption from each line of the captions file and add them to the DataFrame
for caption in captions:
    if(len(captions_df) > 100) : 
      break
    image_id, caption_text = caption.strip().split(',', 1)
    captions_df = captions_df.append({'image_id': image_id, 'caption': caption_text}, ignore_index=True)

captions_df.head(5)

# Image feature extractor
We will use an image model (pretrained on imagenet) to extract the features from each image. The model was trained as an image classifier, but setting `include_top=False` returns the model without the final classification layer, so we can use the last layer of feature-maps:

In [ ]:
IMAGE_SHAPE=(224, 224, 3)
mobilenet = tf.keras.applications.MobileNetV3Small(
    input_shape=IMAGE_SHAPE,
    include_top=False,
    include_preprocessing=True)
mobilenet.trainable=False

In [ ]:
# function to load an image and resize it for the model:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMAGE_SHAPE[:-1])
    return img

`load_image(ex_path)` returns a single image tensor with shape **(height, width, channels)**, where height, width, and channels depend on the image. The `tf.newaxis` expression creates a new dimension of size 1, which turns the image tensor into a batch of size 1 with shape **(1, height, width, channels)**. The `:` expression then selects all elements along the remaining dimensions, effectively creating a batch with a single element that corresponds to the image.

In [ ]:
# The model returns a feature map for each image in the input batch:
ex_path = image_files[0]
test_img_batch = load_image(ex_path)[tf.newaxis, :]

print(test_img_batch.shape)
print(mobilenet(test_img_batch).shape)

# Setup the text tokenizer/vectorizer